In [ ]:
# Importing Libraries
import pandas as pd
import numpy as np
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score



In [ ]:
# Importing the datset 
dataset = pd.read_csv('Synthetic_Financial_datasets_log.csv')
df = dataset # Making a copy of Original Dataset
df.info # Understanding dataset

In [ ]:
# Removing Uncessary Columns
df.drop(['step','isFlaggedFraud','nameOrig','nameDest'], inplace= True, axis = 1)

In [ ]:
# Balancing the fraud and not fraud data
oversample = SMOTE()
x,y = oversample.fit_resample(df[['Cash_IN','Cash_Out','Debit','Payment','Transfer','amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']],df['isFraud'])
X = pd.DataFrame(x)
Y = pd.DataFrame((y))
Balanced_df = pd.concat([X,Y],axis= 1)

In [ ]:
# Feature Engineering
encoder = OneHotEncoder()
a = encoder.fit_transform(df[['type']]).toarray() # Changing Categorical column in to numerical columns
encode_type = pd.DataFrame(a, columns= encoder.get_feature_names_out()) 
df['Cash_IN'],df['Cash_Out'],df['Debit'],df['Payment'],df['Transfer'] = encode_type['type_CASH_IN'],encode_type['type_CASH_OUT'],encode_type['type_DEBIT'],encode_type['type_PAYMENT'],encode_type['type_TRANSFER']
df.drop(['type'],axis=1)

In [ ]:
# Random Forest Model
fraud = Balanced_df['isFraud']
normal = Balanced_df.drop('isFraud',axis=1)
Norm_Train, Norm_Test,Fraud_Train,Fraud_Test = train_test_split(normal,fraud,train_size=0.3,random_state= 42,stratify= y)
rf_model = RandomForestClassifier(n_estimators= 100,n_jobs= -1 ,random_state=42)
rf_model.fit(Norm_Train,Fraud_Train)
Fraud_pred = rf_model.predict(Norm_Test)

In [ ]:
# Testing the accuraqcy of the model
print("Accuracy",accuracy_score(Fraud_Test,Fraud_pred))
print("Confusion MAtrix\n|" , confusion_matrix(Fraud_Test,Fraud_pred))
print("Classification Report", classification_report(Fraud_Test,Fraud_pred)) 

In [ ]:
importances = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)
print("Feature Importances:\n", importances) 


NameError: name 'pd' is not defined